In [ ]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_python_protos.protos.vega as vega_protos


In [ ]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_a1_WALLET = WalletConfig("a1", "pin")

TRADER_a2_WALLET = WalletConfig("a2", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_3_WALLET = WalletConfig("T3", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_2_WALLET, TRADER_3_WALLET, TRADER_a1_WALLET , TRADER_a2_WALLET, TERMINATE_WALLET]


In [ ]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

In [ ]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [ ]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
)

tdai_id = vega.find_asset_id(symbol="tDAI")

mint_amount = {MM_WALLET.name: 10e12, TRADER_a1_WALLET.name: 10e12, TRADER_a2_WALLET.name: 10e12, TRADER_2_WALLET.name: 2000, TRADER_3_WALLET.name: 90, TERMINATE_WALLET.name: 2000}

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=mint_amount[MM_WALLET.name]
)
vega.mint(
    TRADER_a1_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a1_WALLET.name]
)
vega.mint(
    TRADER_a2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_a2_WALLET.name]
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_2_WALLET.name]
)
vega.mint(
    TRADER_3_WALLET.name,
    tdai_id,
    amount=mint_amount[TRADER_3_WALLET.name]
)
vega.mint(
    TERMINATE_WALLET.name,
    tdai_id,
    amount=mint_amount[TERMINATE_WALLET.name]
)

vega.wait_for_total_catchup()


In [ ]:
vega.update_network_parameter(
            MM_WALLET.name, 
            parameter="market.margin.scalingFactors", 
            new_value="{\"search_level\": 1.5, \"initial_margin\": 2, \"collateral_release\": 3}"
        )

In [ ]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=0,
        market_decimals=0,
        risk_aversion=0.000001,
        tau=0.1,
        sigma=1,
        
)

market_id = vega.all_markets()[0].id

In [ ]:
# submit liquidity
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=100000,
    fee=0.001,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=55,
    delta_sell=55,
    is_amendment=False,
)

vega.forward('1s')
vega.forward('1s')

In [ ]:
# Post GTC limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=5,
    price=5,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=1000,
    wait=True,
)

buy_order_id = vega.submit_order(
    trading_wallet=TRADER_a2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=10,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_a1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=10,
    wait=True,
)


In [ ]:
# Check order status/ market state

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))
    print(vega.margin_levels(wallet.name))
    print(general+margin+bond - mint_amount[wallet.name])
    print(vega.order_status(order_id=buy_order_id))

print(vega.get_latest_market_data(market_id=market_id))

In [ ]:
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=40,
    price=50,
    wait=True,
)
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(vega.margin_levels(wallet.name))
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))



In [ ]:

buy_order_id = vega.submit_order(
    trading_wallet=TRADER_3_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=100,
    wait=True,
)

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(vega.margin_levels(wallet.name))
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

In [ ]:
# Check order status/ market state after amend
print(vega.order_status(order_id=buy_order_id))

print(vega.get_latest_market_data(market_id=market_id))